In [8]:
import pandas as pd
from glob import glob
from functools import partial
from types import SimpleNamespace

from lob_seq_model import LobPredModel

In [5]:
data_dir = '/nfs/home/peern/LOBS5/data/'

In [6]:
message_files = glob(data_dir + '*message*')

In [31]:
df = pd.read_csv(
    message_files[0],
    names=['time', 'event_type', 'order_id', 'size', 'price', 'direction'],
    index_col=False)

/nfs/home/peern/rlenv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/nfs/home/peern/rlenv/lib/python3.8/site-packages/pandas/util/_decorators.py:311: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return func(*args, **kwargs)


In [32]:
df

,time,event_type,order_id,size,price,direction
0,34200.000113,4,24695958,134,987500,-1
1,34200.000113,4,25053166,200,987500,-1
2,34200.000113,5,0,200,987500,1
3,34200.000135,5,0,600,987500,1
4,34200.000163,5,0,100,987500,1
...,...,...,...,...,...,...
2244836,57599.996507,1,770732874,1385,979600,1
2244837,57599.996541,1,770732886,1,979600,1
2244838,57599.997147,1,770733066,3500,978700,1
2244839,57599.997521,3,770732874,1385,979600,1


In [26]:
df.iloc[:, -1].dropna()

845        UBSS
919        UBSS
10719      UBSS
13353      UBSS
13395      UBSS
15421      UBSS
66766      UBSS
291238     ETMM
2139765    UBSS
2139766    UBSS
2144425    UBSS
2155614    UBSS
2155628    UBSS
2169474    UBSS
2173455    UBSS
2173806    UBSS
2182265    UBSS
2183214    UBSS
2183462    UBSS
2183671    UBSS
2186136    UBSS
2201236    UBSS
2201259    UBSS
2201272    UBSS
2220728    UBSS
2227472    UBSS
Name: null, dtype: object

In [ ]:
args = SimpleNamespace(
    d_model = 128,  # Number of features, i.e. H, dimension of layer inputs/outputs
    n_layers = 6,  # Number of layers in the network
    activation_fn = 'half_glu1', # 
    p_dropout = 0.,  # probability of dropout
    mode = 'pool',  # options: (for classification tasks) pool: mean pooling \\ last: take last element
    prenorm = True,  # True: use prenorm, False: use postnorm
    batchnorm = True,  # True: use batchnorm, False: use layernorm
    bn_momentum = 0.95,  # batchnorm momentum
)

d_output = 2 + 7 + 6 # categorical: 2, 7, contin: 6

In [ ]:
ssm_init_fn = init_S5SSM(
    H=args.d_model,
    P=ssm_size,
    Lambda_re_init=Lambda.real,
    Lambda_im_init=Lambda.imag,
    V=V,
    Vinv=Vinv,
    C_init=args.C_init,
    discretization=args.discretization,
    dt_min=args.dt_min,
    dt_max=args.dt_max,
    conj_sym=args.conj_sym,
    clip_eigs=args.clip_eigs,
    bidirectional=args.bidirectional)

In [ ]:
model_cls = partial(
    LobPredModel,
    ssm=ssm_init_fn,
    d_output=d_output,
    d_model=args.d_model,
    n_layers=args.n_layers,
    padded=False,
    activation=args.activation_fn,
    dropout=args.p_dropout,
    mode=args.mode,
    prenorm=args.prenorm,
    batchnorm=args.batchnorm,
    bn_momentum=args.bn_momentum,
)